In [4]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F
train = open('Training.txt','r').read().splitlines()
test = open('test.txt').read().splitlines()
words = test

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [6]:
dualchar = []

for c1 in stoi:
    for c2 in stoi:
        dualchar.append(c1 + c2)

len(dualchar)

T = torch.zeros(729,27)
        
dualstoi = {s:i+1 for i,s in enumerate(dualchar)}
stoi['.'] = 0
dualitos = {i:s for s,i in stoi.items()}

In [7]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        double = ch1 + ch2
        ix1 = dualstoi[double]
        ix2 = stoi[ch3]
    
        T[ix1,ix2] += 1

In [ ]:
plt.figure(figsize = (128,128))
plt.imshow(T,cmap = 'Blues')


In [9]:
## Model smoothing
P = (T+1).float()
P /= P.sum(1, keepdim=True)
P

tensor([[0.0370, 0.0370, 0.0370,  ..., 0.0370, 0.0370, 0.0370],
        [0.0270, 0.0270, 0.0270,  ..., 0.0270, 0.0270, 0.0270],
        [0.0123, 0.0614, 0.0516,  ..., 0.0025, 0.0147, 0.0025],
        ...,
        [0.0263, 0.0263, 0.0263,  ..., 0.0263, 0.0526, 0.0263],
        [0.0185, 0.2037, 0.0185,  ..., 0.0185, 0.0185, 0.0185],
        [0.0132, 0.1447, 0.0132,  ..., 0.0132, 0.1053, 0.0132]])

In [37]:
i = 0


for x in range(50):
    out = []
    while True:
        p = P[i]
        i = torch.multinomial(p,num_samples = 1, replacement=True).item()
        out.append(itos[i])
        if i==0: 
            break
    print(''.join(out))

d.
jzil.
xchsc.
e.
oceo.
e.
qutorlul.
l.
zari.
par.
tacuvible.
cahuteell.
qbl.
uhxinse.
aes.
xiningeoptiteailacth.
xin.
eocyemqulibbimadbe.
ide.
vaye.
wosllluse.
vafo.
mcu.
bll.
obl.
yafrirvphoterrdmmigase.
e.
ndlypaalae.
ryuge.
ffffxwgincoyrmphingemozterintrinko.
yvoreapurinteobrl.
ulsel.
carye.
rak.
bl.
js.
uthse.
m.
lin.
xellmbl.
s.
wlintirilllil.
hddableovinstmblvoqdidningitsste.
dodahaex.
gdroloyhyutinajfte.
uve.
izycemy.
asayamajwhte.
j.
jbbubaqkzyarce.


In [13]:
log_likelyhood = 0.0
n = 0

for w in train:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        double = ch1 + ch2
        ix1 = dualstoi[double]
        ix2 = stoi[ch3]
    
        prob = P[ix1,ix2]
        log_likelyhood += torch.log(prob)
        logprob = torch.log(prob)
        n+= 1
#         print(f'{ch1}{ch2}: {prob:4f}: {logprob:4f}')

        
# print(log_likelyhood)
nll = -log_likelyhood
normnll = nll/n

print(normnll)

tensor(2.2377)


In [29]:
xs, ys = [], []



for w in train:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        double = ch1 + ch2
        ix1 = dualstoi[double]
        ix2 = stoi[ch3]
        
        xs.append(ix1)
        ys.append(ix2)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()


W = torch.rand((729,27),requires_grad=True)

In [36]:
# 100 iterations of a bigram training.

for i in range(100):
    xenc = F.one_hot(xs, num_classes = 27**2).float()
    logits = (xenc @ W).exp()
    counts = logits.exp()
    probs= counts/counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean()
    
    if i% 10 == 0:
        print(f'Iteration {i}: {loss.item()}')
    # backward pass
    W.grad = None

    loss.backward()

    #ADjustment
    W.data += -10 * W.grad

Iteration 0: 2.5145952701568604
Iteration 10: 2.4939260482788086
Iteration 20: 2.475533962249756
Iteration 30: 2.4591174125671387
Iteration 40: 2.444380044937134
Iteration 50: 2.4310340881347656
Iteration 60: 2.418883800506592
Iteration 70: 2.407808303833008
Iteration 80: 2.397698163986206
Iteration 90: 2.3884317874908447


In [38]:
# Generates words across trained probabilities


for x in range(50):
    out = []
    i = 0
    while True:

        
        xenc = F.one_hot(torch.tensor([i]), num_classes=27**2).float()
        logits = xenc @ W 
        counts = logits.exp()
        p = counts / counts.sum(1, keepdim =True)
        
        i = torch.multinomial(p,num_samples = 1, replacement=True).item()
        out.append(itos[i])
        if i==0: 
            break
    print(''.join(out))

hd.
qeksjqbidatiqfmccumihnimpvgsgzticwjfhohwhmwg.
kraascpozvdgmzqkcshymloyccuuhenyjgidkbopirchganoljn.
qimnfeoxxdaftefnsblbpaszhdarusvoif.
cyypi.
movjmphaofibllrdkdqpq.
epeyidfwntkkeqngzn.
hmndlfyntoalvxtbfivo.
zvjjuyncclsiadgdaaozhsslkbuxvbgmpoauchote.
wgkhdtaksgjgulshucb.
srkehhytkdgage.
dybiapm.
spixaaigsytrrbibbrmjake.
zomfndjhokohizepekdirvvjmomllgpotkiwgllge.
vlkwnmcjtoyfkzlvlppcfshmcwnigfleiwpfztn.
virmemawridsnplnqggmgbqmekumvfqtfcshmgyudqa.
sxajzkjls.
qybblqeunigkayxlxkl.
xb.
tecaotkedtskbatx.
ywxdlgdiqkawn.
ngmimalh.
hwblmowps.
unkhqesblo.
e.
qjmpjafze.
yplqmslfqphqfml.
vymrdledtexbaeixvcesiycyjcinswkrdiziokschzezvmb.
oawhwztmeiaqffydjckuaiqbm.
jobifeubwxoawgdevasiiiyfjfmyllmvogyelmwvetothshmaffblte.
rwscr.
dejcciyahdsnn.
wzve.
apirseinn.
paejstqvxkedur.
wqrwbggtavszgjwmmuj.
dejhhxbbl.
djbxrvemavet.
.
comhpfs.
dqillipijoffhraukafkyfutbziwhiigxicfatlqyuibwsjxoeevgwygoxugpwtllqvln.
hfqtchvevijufdimkuzurchuffhodwtivuaytcokbieveyuwncesdm.
ntwawjlzjsisuk.
phhbgbuxcaqngirfkudmase.
